# Using iguazio DB and File System with Spark 

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### Read a CSV file from iguazio file system into Spark dataframe

In [3]:
df = spark.read.option("inferSchema", "true").csv('v3io://bigdata/weather.csv')
df.show()

+-----------------+--------+----+-----+----+----+----+----+----+----+----+----+----+----+
|              _c0|     _c1| _c2|  _c3| _c4| _c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|
+-----------------+--------+----+-----+----+----+----+----+----+----+----+----+----+----+
|          STATION|    DATE|PRCP| SNWD|SNOW|TMAX|TMIN|AWND|WDF2|WDF5|WSF2|WSF5|PGTM|FMTM|
|GHCND:USW00094728|20000101|   0|-9999|   0| 100|  11|  26| 250| 230|  72|  94|1337|1337|
|GHCND:USW00094728|20000102|   0|-9999|   0| 156|  61|  21| 260| 260|  72| 112|2313|2314|
|GHCND:USW00094728|20000103|   0|-9999|   0| 178| 106|  30| 260| 250|  67|  94| 320| 321|
|GHCND:USW00094728|20000104| 178|-9999|   0| 156|  78|  35| 320| 350|  67| 107|1819|1840|
|GHCND:USW00094728|20000105|   0|-9999|   0|  83| -17|  51| 330| 340| 107| 143| 843| 844|
|GHCND:USW00094728|20000106|   0|-9999|   0|  56| -22|  30| 220| 250|  67|  98|1833|1834|
|GHCND:USW00094728|20000107|   0|-9999|   0|  94|  17|  42| 300| 310| 103| 156|1521|1601|
|GHCND:USW

### Write the dataframe into iguazio Key/Value DB API

In [4]:

df.write.format("io.iguaz.v3io.spark.sql.kv").mode("append").option("key", "_c0").save("v3io://bigdata/weather_table")

### Read the data from the DB with predicate offload (push-down)

In [5]:
spark.read.format("io.iguaz.v3io.spark.sql.kv").load("v3io://bigdata/weather_table").show()

+-----------------+--------+----+-----+----+----+----+----+----+----+----+----+----+----+
|              _c0|     _c1| _c2|  _c3| _c4| _c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|
+-----------------+--------+----+-----+----+----+----+----+----+----+----+----+----+----+
|GHCND:USW00094728|20000101|   0|-9999|   0| 100|  11|  26| 250| 230|  72|  94|1337|1337|
|          STATION|    DATE|PRCP| SNWD|SNOW|TMAX|TMIN|AWND|WDF2|WDF5|WSF2|WSF5|PGTM|FMTM|
+-----------------+--------+----+-----+----+----+----+----+----+----+----+----+----+----+

